In [1]:
# imports and loading
import pandas as pd
import numpy as np
from scipy.optimize import minimize

saved_cActivators = pd.read_pickle('../data/saved_cActivator_vals.pkl')
saved_cInhibitors = pd.read_pickle('../data/saved_cInhibitor_vals.pkl')

# depreceated

In [ ]:
# we want element-wise comparison

# settings
variance_penalty_mult = 1

# Define the objective function that we want to minimize. 
# In this case, it's the Frobenius norm of (A*B - C).
def objective(x, nrows_A, ncols_A, nrows_B, ncols_B, C):
    # Reshape the input to the correct dimensions
    A = x[:nrows_A*ncols_A].reshape(nrows_A, ncols_A)
    B = x[nrows_A*ncols_A:].reshape(nrows_B, ncols_B)
    
    err = np.linalg.norm(A * B - C)
    
    # calculate penalty
    penalty = 0
    # add variance of every row of A and column of B
    for col_ct in range(ncols_A):
        penalty += np.var(A[:, col_ct])
    for row_ct in range(nrows_B):
        penalty += np.var(B[row_ct, :])

    return(err + variance_penalty_mult*penalty)

# Callback function to print the current error
def print_error(x):
    current_error = objective(x, nrows, ncols, C)
    print("Current error:", current_error)
    
# Initialize the dimensions of A and B.
nrows_A, ncols_A = np.shape(saved_cActivators)  # Dimensions for A
nrows_B, ncols_B = np.shape(saved_cActivators)  # Dimensions for B
C = saved_cActivators


# Define the constraints on A and B. In this case, we're assuming that all values must be non-negative.
bounds = [(0, None) for _ in range(nrows_A * ncols_A + nrows_B * ncols_B)]

# Initialize A and B randomly
A_init = np.random.rand(nrows_A, ncols_A)
B_init = np.random.rand(nrows_B, ncols_B)

# Flatten A and B to turn them into 1D arrays, then concatenate them to get our initial guess
x0 = np.concatenate([A_init.flatten(), B_init.flatten()])

# Run the optimization
res = minimize(objective, x0, args=(nrows_A, ncols_A, nrows_B, ncols_B, C), bounds=bounds, callback=print_error)

# Extract the optimized A and B
A_opt = res.x[:nrows_A*ncols_A].reshape(nrows_A, ncols_A)
B_opt = res.x[nrows_A*ncols_A:].reshape(nrows_B, ncols_B)

print('success!')
